In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)


Mounted at /content/drive


In [2]:
import torch
import torchvision
import PIL
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from pathlib import Path
from PIL import Image

In [3]:
path = Path(r"/content/drive/MyDrive/AdClassification/Ad_Classification")
train_dir = path / "train"
test_dir = path / "test"

train_dir, test_dir

(PosixPath('/content/drive/MyDrive/AdClassification/Ad_Classification/train'),
 PosixPath('/content/drive/MyDrive/AdClassification/Ad_Classification/test'))

In [4]:
train_transform = transforms.Compose(
    [transforms.Resize(size = (250, 250)),
    transforms.RandomRotation(degrees= 45),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor()])

test_transform = transforms.Compose(
    [transforms.Resize(size = (250, 250)),
    transforms.RandomRotation(degrees= 45),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor()])


In [5]:
train_data =  ImageFolder(root = train_dir, transform=train_transform, target_transform=None)
test_data =  ImageFolder(root = test_dir, transform=test_transform, target_transform=None)

In [6]:
train_data, test_data

(Dataset ImageFolder
     Number of datapoints: 97
     Root location: /content/drive/MyDrive/AdClassification/Ad_Classification/train
     StandardTransform
 Transform: Compose(
                Resize(size=(250, 250), interpolation=bilinear, max_size=None, antialias=True)
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 23
     Root location: /content/drive/MyDrive/AdClassification/Ad_Classification/test
     StandardTransform
 Transform: Compose(
                Resize(size=(250, 250), interpolation=bilinear, max_size=None, antialias=True)
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ))

In [ ]:
train_data.classes, train_data.class_to_idx

(['Creative', 'Non_Creative'], {'Creative': 0, 'Non_Creative': 1})

In [ ]:
data_loader = DataLoader(dataset=train_data, batch_size = 1, shuffle = True)


In [7]:
train_loader = DataLoader(dataset=train_data, batch_size = 1, shuffle = True)
test_loader = DataLoader(dataset=test_data, batch_size = 1, shuffle = False)

In [ ]:
images, labels = next(iter(train_loader))

In [ ]:
images[0], labels[0]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor(1))

In [ ]:
images[0].shape

torch.Size([3, 250, 250])

In [8]:
import torch.nn as nn

In [9]:
class AdClassification(nn.Module):

    def __init__(self, input_shape : int, hidden_shape:int, output_shape : int) -> None:
        super(AdClassification, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_shape, out_channels=hidden_shape, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_shape*62*62*10, out_features=output_shape)
        )

    def forward(self, x:torch.Tensor):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.classifier(x2)
        return x3


In [10]:
model = AdClassification(3, 10, len(train_data.classes))

model.state_dict()

OrderedDict([('block1.0.weight',
              tensor([[[[ 0.1820,  0.0088,  0.1225],
                        [-0.1496,  0.0132,  0.0261],
                        [-0.1414, -0.0766, -0.0159]],
              
                       [[ 0.1090, -0.0170,  0.1077],
                        [ 0.1360,  0.1390, -0.1017],
                        [-0.0896,  0.1452,  0.0122]],
              
                       [[ 0.1617,  0.1511, -0.0010],
                        [ 0.0482, -0.1772, -0.1638],
                        [ 0.1898,  0.1487,  0.0761]]],
              
              
                      [[[-0.0641, -0.0008,  0.1467],
                        [-0.0704, -0.0157, -0.1911],
                        [ 0.1086,  0.0269,  0.1067]],
              
                       [[-0.0314,  0.1068,  0.0757],
                        [-0.1663, -0.0010,  0.1021],
                        [-0.1370, -0.0753, -0.1328]],
              
                       [[-0.0216,  0.1311,  0.1756],
                       

In [ ]:
model.parameters()

<generator object Module.parameters at 0x00000262CAB60AC0>

In [11]:
optimizer  = torch.optim.SGD(params = model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

In [12]:
def train_step(model: nn.Module,
               train_loader : torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(train_loader):

        pred_y = model(X)
        loss = loss_fn(pred_y, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_class = torch.argmax(torch.softmax(pred_y, dim = 1), dim = 1)
        train_acc += (pred_class == y).sum().item()/len(pred_y)

    train_loss = train_loss/len(train_loader)
    train_acc = train_acc/len(train_loader)

    return train_loss, train_acc

In [13]:
def test_step(
        model: nn.Module,
        test_loader : torch.utils.data.DataLoader,
        loss_fn : torch.nn.Module):

    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():

        for batch, (X,y) in enumerate(test_loader):

            pred_y = model(X)
            loss = loss_fn(pred_y, y)
            test_loss += loss.item()

            pred_class = torch.argmax(torch.softmax(pred_y, dim =1), dim =1)
            test_acc += (pred_class == y).sum().item()/len(pred_y)

        test_loss = test_loss/len(test_loader)
        test_acc = test_acc/len(test_loader)

        return test_loss, test_acc

In [16]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_loader: torch.utils.data.DataLoader,
          test_loader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 250):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           train_loader=train_loader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)

        test_loss, test_acc = test_step(model=model,
            test_loader=test_loader,
            loss_fn=loss_fn)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [17]:
train(model, train_loader, test_loader, optimizer, loss_fn)

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5744 | train_acc: 0.7216 | test_loss: 0.8754 | test_acc: 0.5217
Epoch: 2 | train_loss: 0.5515 | train_acc: 0.7010 | test_loss: 0.8005 | test_acc: 0.5217
Epoch: 3 | train_loss: 0.5609 | train_acc: 0.6907 | test_loss: 0.7413 | test_acc: 0.5217
Epoch: 4 | train_loss: 0.5693 | train_acc: 0.7010 | test_loss: 0.7700 | test_acc: 0.5652
Epoch: 5 | train_loss: 0.5845 | train_acc: 0.7113 | test_loss: 0.7060 | test_acc: 0.6087
Epoch: 6 | train_loss: 0.5792 | train_acc: 0.6186 | test_loss: 0.6993 | test_acc: 0.4348
Epoch: 7 | train_loss: 0.5431 | train_acc: 0.7423 | test_loss: 0.9925 | test_acc: 0.4783
Epoch: 8 | train_loss: 0.5762 | train_acc: 0.7526 | test_loss: 0.7567 | test_acc: 0.5652
Epoch: 9 | train_loss: 0.5441 | train_acc: 0.7629 | test_loss: 0.8902 | test_acc: 0.5217
Epoch: 10 | train_loss: 0.5415 | train_acc: 0.7216 | test_loss: 0.7934 | test_acc: 0.6087
Epoch: 11 | train_loss: 0.5697 | train_acc: 0.7113 | test_loss: 0.9897 | test_acc: 0.6087
Epoch: 12 | train_l

{'train_loss': [0.5743812395970231,
  0.5514865643968901,
  0.5609219374192744,
  0.5693082119597449,
  0.5844649460896388,
  0.5791809073620543,
  0.5430788053871737,
  0.5761698553549874,
  0.5440621442094291,
  0.5414628091066614,
  0.5696842328299644,
  0.5862159935575094,
  0.5439913166955728,
  0.5655632352398843,
  0.5338119324872789,
  0.5120903970193618,
  0.5268239961437804,
  0.5471382071216082,
  0.5547031386918628,
  0.5370194090472669,
  0.4982926305575469,
  0.5459581075497836,
  0.5320607334298572,
  0.5478463916434455,
  0.5294556242519433,
  0.5110367648255503,
  0.5166944772696372,
  0.5441514288660946,
  0.5320986892989615,
  0.48060539450266004,
  0.5181492249548589,
  0.47361670858850796,
  0.5345172392607657,
  0.47975719487294555,
  0.4862235737388435,
  0.5205660975855035,
  0.49954618383967075,
  0.4937903800062336,
  0.5063225520107434,
  0.5246213478775522,
  0.46185319962882504,
  0.4770203634458066,
  0.5235140219225174,
  0.5018971941827499,
  0.483876718

In [18]:
Model_path = Path("AdClassificationModel")
Model_path.mkdir(parents = True, exist_ok = True)

Model_Name  = "AdDetection.pt"

SaveModelPath = Model_path / Model_Name

SaveModelPath

PosixPath('AdClassificationModel/AdDetection.pt')

In [19]:
torch.save(model.state_dict(), SaveModelPath)

In [32]:
from PIL import Image
import cv2

img = cv2.imread(r"/content/drive/MyDrive/AdClassification/Ad_Classification/train/Creative/Corona.jpeg")
pic = Image.open(r"/content/drive/MyDrive/AdClassification/Ad_Classification/train/Creative/Corona.jpeg")
test_image = train_transform(pic)


test_image.shape, sample.shape

(torch.Size([3, 250, 250]), torch.Size([3, 1020, 736]))

In [26]:
imported = model

imported.load_state_dict(torch.load(SaveModelPath))

<All keys matched successfully>

In [35]:
imported.eval()

with torch.inference_mode():
    loaded_pred = imported(test_image.unsqueeze(dim =0))
    pred_class = torch.argmax(torch.softmax(loaded_pred, dim =1), dim =1)
pred_class


tensor([1])

In [ ]:

#& Strategies to Improve the Misclassifications

"""
1. Increasing the data size will reduce the misclassification
2. Data Augmentation - In the above code we can include some more transformations.
   This can be using torchvision.transforms function.This can be implemented in the
   above coding
3. Rather than going for custom model, we can go for Transfer Learning. We can take
   Pre-trained model like VGG, Mobilenet etc.
4. We can use learning rate decay
5. Early Stopping - This will help us stop the training process when the training reaches
   required amount of loss and   accuracy.
6. Add More units of Sequential Layers Blocks, Increase the number of hidden layers,
   Change the activation functions.
7. Tweak Learning Rate migh also help achieve the goal
8. Vary the Epoch sizes - Increasing the Epoch size shall help the model find the global minima.

"""